In [3]:
# 02:06:35

# LangChain: Models, Prompts และ Output Parsers


## เนื้อหา

 * การเรียกใช้ API ของ OpenAI โดยตรง
 * การเรียกใช้ API ผ่าน LangChain
   * Prompts
   * Models
   * Output parsers

In [3]:
!pip install openai==0.28

In [4]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_KEY')

## Chat API : OpenAI
ต่อไปนี้จะเป็นการเรียนใช้ API จาก OpenAI โดยตรง
(ซึ่งเป็นสิ่งที่เราใช้ในบทที่แล้ว)

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [
        {
        "role":"user",
         "content":prompt
        }
    ]
    response = openai.ChatCompletion.create(
        model       = model,
        messages    = messages,
        temperature = 0
    )
    return response.choices[0].message["content"]

In [7]:
get_completion("What is 1+1")

'1+1 equals 2.'

In [8]:
customer_email = """
Arrr, I be fuming that me blender lib \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
style = """American English\
in a calm and respectful tone
"""
# prompt = Context + customer_email + style
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""
print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American Englishin a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lib flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
respones = get_completion(prompt)

In [10]:
respones

"I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

<hr>

In [5]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## Chat API : LangChain

เมื่อใช้งานผ่าน interface ของ LangChain จะสามารถทำได้โดยขั้นตอนเหล่านี้

In [12]:
!pip install langchain==0.0.232

### Model

เป็นการกำหนดว่าต้องการใช้ Model LLM ตัวไหนในการใช้งาน LangChain
LangChain มีการ support Model ต่าง ๆ มากมายไม่ว่า OpenAI, Cohere, Claude หรือแม้แต่โมเดลจาก Huggingface และยังมีการเพิ่มเติมเข้าไปเรื่อย ๆ

ในเนื้อหาบทนี้เราจะใช้งานโมเดลของ OpenAI โดยเฉพาะ gpt-3.5-turbo เป็นหลัก

In [13]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)
chat

### Prompt template
- เป็น Interface ในการกำหนด Prompt ภายใน LangChain โดยเราสามารถกำหนดเป็น Template String ได้
- อีกทั้งถ้าหากต้องการนำตัวแปรไปใส่ เราสามารถครอบ {} ใน template นั้นได้

In [15]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [16]:
from langchain.prompts import ChatPromptTemplate

In [17]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [18]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [19]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [20]:
customer_style = """American English \
in a calm and respectful tone
"""

In [21]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty donn't cover the cost of \
cleaning up me kitchen. I need yer help\
right now, matey!
"""

In [22]:
customer_messages = prompt_template.format_messages(
    style = customer_style,
    text  = customer_email
)

In [24]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty donn't cover the cost of cleaning up me kitchen. I need yer helpright now, matey!\n```\n", additional_kwargs={}, example=False)]

In [23]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [25]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty donn't cover the cost of cleaning up me kitchen. I need yer helpright now, matey!\n```\n" additional_kwargs={} example=False


In [27]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
customer_response

AIMessage(content="Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy!", additional_kwargs={}, example=False)

In [28]:
print(customer_response.content)

Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy!


## Output Parsers

เป็นวิธีการกำหนดว่าต้องการให้ผลลัพท์ออกมาในลักษณะใด (ในตัวอย่างเราต้องการให้ผลลัพท์ออกมาในรูปแบบ json string)

In [29]:
{
    "gift":False,
    "delivery_days":5,
    "price_value":"pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [30]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""


In [31]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [32]:
messages = prompt_template.format_messages(text=customer_review)
chat     = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [33]:
type(response.content)

str

In [34]:
# บรรทัดนี้ควรจะ Error เนื่องจากผลลัพท์เป็น JSON string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse ผลลัพท์จากโมเดลให้ออกมาเป็น structured type (Dictionary)

In [35]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [37]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [38]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [41]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')])

In [39]:
format_instructions = output_parser.get_format_instructions()

In [40]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [42]:
review_template_2 = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template = review_template_2)
messages = prompt.format_messages(
                                text=customer_review,
                                format_instructions=format_instructions
                                )

In [43]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: This leaf blower is pretty amazing. It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.




In [44]:
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [45]:
output_dict = output_parser.parse(response.content)
output_dict.get('delivery_days')

2

<hr>

# LangChain: Memory

## เนื้อหา
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [47]:
llm = ChatOpenAI(temperature = 0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [48]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [49]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [50]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know or discuss?'

In [51]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew. Is there anything else you would like to know or discuss?


In [52]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know or discuss?"}

In [54]:
print(memory.load_memory_variables({}))

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know or discuss?"}


In [55]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input":"Hi"},
    {"output":"What's up"}
)
print(memory.buffer)

Human: Hi
AI: What's up


In [57]:
memory.load_memory_variables({})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

<hr>

# ConversationBufferWindowMemory

In [58]:
from langchain.memory import ConversationBufferWindowMemory

In [59]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input":"Hi"},{"output":"What's up"})
memory.save_context({"input":"Not much, just hanging"},{"output":"Cool"})

In [60]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [61]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm = llm,
    memory=memory,
    verbose=True
)

In [62]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [63]:
conversation.predict(input="What is 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [64]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

In [67]:
!pip install tiktoken

In [68]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

In [69]:
llm = ChatOpenAI(temperature=0.0)

In [70]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input":"AI is what?!"},
                    {"output":"Amazing!"})
memory.save_context({"input":"Backpropagation is what?"},
                    {"output":"Beautiful!"})
memory.save_context({"input":"Chatbots are what?"},
                    {"output":"Charming!"})

In [71]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [72]:
from langchain.memory import ConversationSummaryBufferMemory

In [73]:
# create a long string

In [74]:
schedule = "There is a meeting at 8am with your product team.\
You will need your powerpoint presentation prepared.\
9am-12pm have time to work on your LangChain\
project which will go quickly because Langchain is such a powerful tools.\
At Noon, lunch at the italian resturant with a customer who is driving\
from over an hour away to meet you to understand the latest AI.\
Be sure to bring your laptop to show the latest LLM demo."

In [75]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input":"Hello"},{"output":"What's up"})
memory.save_context({"input":"Not much, just hanging"},
                    {"output":"Cool"})
memory.save_context({"input":"What is on the schedule today?"},
                    {"output":f"{schedule}"})

In [76]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI with a hello. The AI responds with a casual "What\'s up" and the human mentions they are just hanging. The AI acknowledges this with a "Cool" and the human inquires about the schedule for the day.\nAI: There is a meeting at 8am with your product team.You will need your powerpoint presentation prepared.9am-12pm have time to work on your LangChainproject which will go quickly because Langchain is such a powerful tools.At Noon, lunch at the italian resturant with a customer who is drivingfrom over an hour away to meet you to understand the latest AI.Be sure to bring your laptop to show the latest LLM demo.'}

In [78]:
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [79]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI with a hello. The AI responds with a casual "What's up" and the human mentions they are just hanging. The AI acknowledges this with a "Cool" and the human inquires about the schedule for the day.
AI: There is a meeting at 8am with your product team.You will need your powerpoint presentation prepared.9am-12pm have time to work on your LangChainproject which will go quickly because Langchain is such a powerful tools.At Noon, lunch at the italian resturant with a customer who is drivingfrom over an hour away to meet you to understand the latest AI.Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to 

'You could showcase the language model capabilities of LangChain, such as its ability to generate natural language text, perform language translation, or even summarize text. Additionally, you could demonstrate how LangChain can be integrated into various applications or platforms to enhance their language processing capabilities.'

In [80]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI with a hello. The AI responds with a casual "What\'s up" and the human mentions they are just hanging. The AI acknowledges this with a "Cool" and the human inquires about the schedule for the day. The AI informs the human of a meeting at 8am with the product team, followed by work on the LangChain project from 9am-12pm. Lunch at an Italian restaurant with a customer is scheduled for noon, where the latest AI demo will be shown.\nHuman: What would be a good demo to show?\nAI: You could showcase the language model capabilities of LangChain, such as its ability to generate natural language text, perform language translation, or even summarize text. Additionally, you could demonstrate how LangChain can be integrated into various applications or platforms to enhance their language processing capabilities.'}

<hr>

# Chains ใน LangChain

## เนื้อหา

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

## LLMChain

In [81]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [82]:
llm = ChatOpenAI(temperature=0.9)

In [87]:
prompt = ChatPromptTemplate.from_template(
    "Whai is the best name to describe \
    a company that makes {product}?"
)

In [88]:
chain = LLMChain(llm=llm, prompt=prompt)

In [89]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Linens"'

## SimpleSequentialChain

In [90]:
from langchain.chains import SimpleSequentialChain

In [92]:
llm = ChatOpenAI(temperature=0.9)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [93]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [94]:
overall_simple_chain = SimpleSequentialChain(
                            chains  = [chain_one, chain_two],
                            verbose = True
)

In [95]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Regal Linens"
Regal Linens offers premium quality bedding and linens for a luxurious night's sleep, combining comfort, style, and durability.

> Finished chain.


"Regal Linens offers premium quality bedding and linens for a luxurious night's sleep, combining comfort, style, and durability."

## SequentialChain

In [96]:
from langchain.chains import SequentialChain

In [97]:
llm = ChatOpenAI(temperature = 0.9)
# prompt templaate 1 : translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# Chain 1: input = Review and output = English_Review
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    output_key = "English_Review"
)

In [98]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input = English_Review and output = summary
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="summary"
)

In [99]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input = Review and output = language
chain_three = LLMChain(llm=llm, prompt=third_prompt,output_key="language")

In [100]:
# prompt template 4 : follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4 : input = summary, language and output = followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key = "followup_message")

In [101]:
# overall_chain: input = Review
# and output = English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review","summary","followup_message"],
    verbose = True
)

In [102]:
review = """
สินค้าส่งโคตรไว ประมาณ2วันถึงมือเลย ห่อมาหนาแน่นดีมากๆ น่าจะมาจากเกาหลี แต่เครื่องผลิตที่จีน หลังจากได้ของแล้วอย่าลืมลงทะเบียนรับประกัน 2 ปีกับไมโครซอฟด้วย \
เล่นเกมมาแล้ว 3 วัน ปกติดี ซื้อเกมง่ายกว่าที่คิด ไม่ติด oops เลยตัวเครื่องร้อนเวลาเล่นไปนานๆ แต่ไม่มาก ข้อเสียงอย่างเดียวคือติดตั้งเกมนานไปนิด
"""
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nสินค้าส่งโคตรไว ประมาณ2วันถึงมือเลย ห่อมาหนาแน่นดีมากๆ น่าจะมาจากเกาหลี แต่เครื่องผลิตที่จีน หลังจากได้ของแล้วอย่าลืมลงทะเบียนรับประกัน 2 ปีกับไมโครซอฟด้วย เล่นเกมมาแล้ว 3 วัน ปกติดี ซื้อเกมง่ายกว่าที่คิด ไม่ติด oops เลยตัวเครื่องร้อนเวลาเล่นไปนานๆ แต่ไม่มาก ข้อเสียงอย่างเดียวคือติดตั้งเกมนานไปนิด\n',
 'English_Review': 'The product was delivered very quickly, in about 2 days. It was packaged very securely, most likely from Korea but manufactured in China. After receiving the item, remember to register for the 2-year warranty with Microsoft. I have been playing games for 3 days now and everything is working well. Buying games is easier than I thought, no oops errors. The device does get a little hot when playing for a long time, but not too much. The only downside is that installing games takes a bit longer.',
 'summary': 'The product was delivered quickly and securely, with the device working well for gaming, although it does get hot and installing games can be time-cons

## Router Chain

In [104]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [105]:
prompt_infos = [
    {
        "name":"physics",
        "description":"Good for answering questions about physics",
        "prompt_template":physics_template
    },
    {
        "name":"math",
        "description":"Good for answering math questions",
        "prompt_template":math_template
    },
    {
        "name":"History",
        "description":"Good for answering history quesitons",
        "prompt_template":history_template
    },
    {
        "name":"computer science",
        "description":"Good for answering computer science question",
        "prompt_template":computerscience_template
    }
]

In [106]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [107]:
llm = ChatOpenAI(temperature=0)

In [109]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [110]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain  = LLMChain(llm=llm, prompt=default_prompt)

In [111]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:12: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-2160379643.py:12: SyntaxWarning: invalid escape sequence '\ '
  "destination": string \ name of the prompt to use or "DEFAULT"


In [112]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)
router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [113]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True
                        )

In [114]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.12/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [115]:
chain.run("whai is 2+2")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.12/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'what is 2+2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2+2" is 4. This is because when you add 2 and 2 together, you get a total of 4.'

In [117]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

<hr>

# LangChain: Q&A บน Documents (ถาม-ตอบ)

หนึ่งใน Use Case ยอดฮิตของ LLM คือการนำ model ไปอ่านเอกสารที่เราต้องการ กลั่นกรองเนื้อหา และเป็นตัวกรองสำหรับเอกสารนั้น ๆ

ในบทนี้เราจะพาทุกท่ามาเรียนรู้ วิธีในการ Q&A บนข้อมูลของเราเองโดยการใช้ LLM

โดยการทำงานจะประกอบไปด้วย
1. การอ่านข้อมูลทั้งหมด -> Data Loading
2. แบ่งข้อมูลเป็น Chunk ขนาดที่ LLM รับได้ -> Splitting
3. Vectorized Data ทั้งหมด (แปลงเป็น Embedding) แล้ว Store เก็บไว้ -> Storage

หลังจาก Vectorized Data ทั้งหมด เราสามารถสร้างระบบ Query หาข้อมูลที่ต้องการ จากข้อมูลที่ Vectorized ได้ โดยการใช้งาน Similarity Search

4. ป้อนข้อมูล Batch ที่ Query มาให้ AI (ผ่าน Prompt) -> Retrieval
5. จากนั้นจึงเอาไป prompt กับ LLM -> Output

ซึ่งขั้นตอนข้อทั้งหมด LangChain provide Chain ชื่อ VectorstoreIndexCreator ที่รวบขั้นตอนทั้งหมดให้เราเรียบร้อยแล้ว

In [139]:
pip install "langchain==0.0.171" "docarray<0.22"


  Using cached docarray-0.21.1.tar.gz (658 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.3 MB/s eta 0:00:00
  Created wheel for docarray: filename=docarray-0.21.1-py3-none-any.whl size=722056 sha256=3b657b1fd670acd5c3b11ef3be19e662be3f032a48bb8652a25f265b3f02f2a4
  Stored in directory: /root/.cache/pip/wheels/9e/6c/24/4a63e1fc0779237c98bb8175e8d975e657dfd1c084d7114895
Successfully built docarray
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.232
    Uninstalling langchain-0.0.232:
      Successfully uninstalled langchain-0.0.232
  Attempting uninstall: docarray
   

In [6]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])
print(index.query("What is task decomposition?"))


ValueError: To use the DocArrayHnswSearch VectorStore the docarray version >=0.31.0 is expected, received: 0.21.1.To upgrade, please run: `pip install -U docarray`.

<hr>

# LangChain: Agents

Agent เป็นหนึ่งใน Experiment ที่โด่งดังที่สุด และติดเทรนด์ใยช่วงนี้ เครื่องมือหลายๆอย่าง เช่น AutoGPT, babyAGI ล้วนมีลักษณะคล้ายคลึงกับ Agent

โดยสิ่งที่ Agent สามารถทำได้คือ 1. รับคำสั่ง 2. วางแผน 3. ปฎิบัติตามคำสั่ง 4. เรียนรู้ เพื่อที่นำไปปฎิบัติต่อไป

เมื่อมีโครงสร้างพื้นฐานดังกล่าวแล้ว Agent จึงเป็นมากกว่า Chatbot ธรรมดาที่เพียงแค่ตอบคำถามจากสิ่งที่เราต้องการ แต่มันสามารถสังเกต วางแผน และเรียนรู้ที่จะทำงานให้ออกมาดีที่สุด

สิ่งหนึ่งที่ทำให้ความสามารถของ Agent มีสูงมาก และประสิทธิภาพสูงมาก คือ Tools หรือเครื่องมือต่าง ๆ ที่ Agent สามารถใช้ได้ ไม่ว่าจะเป็นการใช้งาน Search Engine (DuckDuckGo) หรืออ่านข้อความจาก Wikipedia หรือแม้แต่ Tools ที่เราต้องการสร้างเองก็ได้

## เนื้อหา:

* ใช้งาน Built-in tools ใน LangChain : DuckDuckGo search, Wikipedia
* สร้าง Tools เอง

In [7]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=1fff391d50f096a2faa9e0255feaba6d178bafa0798fc0373bc8ecb680d196b5
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [9]:
llm = ChatOpenAI(temperature=0)

In [10]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [11]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [12]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Question: What is the 25% of 300?
Thought: We can use a calculator to find the answer to this math question.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% * 300"
}
```

Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [13]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

## Python Agent

หนึ่งใน Agent ที่ทรงพลังที่สุด เนื่องจากมันสามารถ Execute code ได้

In [14]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [15]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [16]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
We can use the `sorted()` function in Python to sort the list of customers based on last name first and then first name.
Action: Python REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]


Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### ดูการทำงานของ Agent

In [17]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False



> Entering new AgentExecutor chain...
We can use the `sorted()` function in Python to sort the list of customers based on last name first and then first name.
Action: Python REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]


Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


[link text](https://)## สร้าง tool แบบ Custom

In [18]:
from langchain.agents import tool
from datetime import date

In [19]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [20]:
agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Warning**:

เนื่องจากปัจจุบัน Agent ยังอยู่ในช่วงเริ่มต้นของการพัฒนา อาจจะมีบางครั้งที่ได้ผลลัพท์ที่อาจจะไม่ถูกต้อง

In [21]:
try:
    result = agent("whats the date today?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to find out today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2025-10-02
Thought:Final Answer: 2025-10-02

> Finished chain.
